In [1]:
import pandas as pd

df_merge = pd.read_csv('data/데이터병합.csv', sep=',', encoding='cp949')
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   터미널명    283 non-null    object 
 1   주소      282 non-null    object 
 2   등록일자    282 non-null    object 
 3   위도      282 non-null    float64
 4   경도      282 non-null    float64
 5   번호      283 non-null    int64  
 6   보관대수    283 non-null    int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 15.6+ KB


In [2]:
df_path = pd.read_excel('data/path/path_count4.xlsx', engine='openpyxl')
df_path.head()
df_path.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19458 entries, 0 to 19457
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   경로      19458 non-null  object
 1   new 출발  19458 non-null  int64 
 2   new 도착  19458 non-null  int64 
 3   개수      19458 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 608.2+ KB


In [3]:
df_path = df_path[df_path['개수']>=500]

In [5]:
extract_col = df_merge[['번호', '위도', '경도']]
terminal_dict = dict()
for num, lat, long in zip(extract_col['번호'], extract_col['위도'], extract_col['경도']):
    terminal_dict[int(num)] = [lat, long]

In [6]:
diffPath = []
for path, count in zip(df_path['경로'], df_path['개수']):
    if count >= 1100:
        weight = 8
    elif count >= 900:
        weight = 6
    elif count >= 700:
        weight = 4
    else:
        weight = 2
        
    sp_lst = path.split('~')
    if sp_lst[0] != sp_lst[1]:
            diffPath.append([int(sp_lst[0]), int(sp_lst[1]), weight])

In [7]:
diffPath

[[18, 27, 6],
 [118, 222, 4],
 [222, 118, 4],
 [27, 18, 4],
 [263, 27, 2],
 [27, 263, 2],
 [222, 272, 2],
 [18, 263, 2],
 [34, 35, 2],
 [263, 18, 2],
 [34, 220, 2],
 [58, 38, 2],
 [27, 109, 2],
 [41, 42, 2],
 [41, 11, 2]]

In [8]:
import folium

In [9]:
df_merge = df_merge.dropna(axis=0)

In [10]:
from folium.plugins import MarkerCluster


m = folium.Map(
    location=[35.2538, 128.6402],
    zoom_start=11
)

coords1 = df_merge[['번호','위도', '경도']]


marker_cluster1 = MarkerCluster().add_to(m)

for name, lat, long in zip(coords1['번호'], coords1['위도'], coords1['경도']):
    folium.Marker([lat, long], popup= [name], icon = folium.Icon(color="green")).add_to(marker_cluster1)

In [11]:
for depart_num, arrive_num, weight in diffPath:
    folium.PolyLine(
        locations = [terminal_dict[depart_num],terminal_dict[arrive_num]],
        tooltip = '{}->{}, 가중치: {}'.format(depart_num, arrive_num, weight),
        weight= weight
    ).add_to(m)

In [12]:
m